# Build the Neural Network

In [1]:
print("Building our First Neural Network")

Building our First Neural Network


A neural network is a module itself that consists of other modules(layers). Every modules in Pytorch subclasses the `nn.Module`.

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
# Set the accelerator for training the model
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


- Define the neural network by subclassing nn.Module and initialize the neural network layers in the `__init__`. 

- Every nn.Module subclass implements the operations on input data in the forward method.

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Create an instance of the NeuralNetwork and move it to the device and print it's structure

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's forward along with some background operations. Do not call `model.forward()` directly!

In [11]:
X = torch.rand(1,28,28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='mps:0')


In [12]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten

- initialize and use nn.Flatten layer to convert each d 28x28 image into a contiguous array of 784 pixel values.

In [14]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear

nn.Linear, the linear layer is a module that applies a linear transformation on the input using it's stored weights and biases.

In [19]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())
print(hidden1)

torch.Size([3, 20])
tensor([[ 0.1192,  0.0855, -0.6055, -0.1172, -0.0813, -0.1265, -0.4832, -0.1372,
         -0.1229,  0.2470,  0.0089,  0.0384, -0.0981, -0.3041,  0.2871, -0.4814,
         -0.4479, -0.0800, -0.1748, -0.1271],
        [-0.2083,  0.1636, -0.7108, -0.1456,  0.1213, -0.3079, -0.0879, -0.2617,
          0.0130,  0.1749, -0.2508,  0.2043, -0.1474, -0.3024,  0.5385, -0.2021,
         -0.8868, -0.1065, -0.3374, -0.5472],
        [ 0.0138, -0.1285, -0.7495, -0.2676,  0.1638,  0.1955, -0.2730,  0.0480,
         -0.0216,  0.1833, -0.1482,  0.2012, -0.1058, -0.2655, -0.2031, -0.4648,
         -0.5155, -0.2494, -0.2936, -0.1638]], grad_fn=<AddmmBackward0>)


#### nn.ReLU

Non Linear activations are what create the complex mappings between the models inputs and outputs. They are applied after linear transformations to introduce non-linearity, helping neural networks learn a wide variety of phenomena.

In [20]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1192,  0.0855, -0.6055, -0.1172, -0.0813, -0.1265, -0.4832, -0.1372,
         -0.1229,  0.2470,  0.0089,  0.0384, -0.0981, -0.3041,  0.2871, -0.4814,
         -0.4479, -0.0800, -0.1748, -0.1271],
        [-0.2083,  0.1636, -0.7108, -0.1456,  0.1213, -0.3079, -0.0879, -0.2617,
          0.0130,  0.1749, -0.2508,  0.2043, -0.1474, -0.3024,  0.5385, -0.2021,
         -0.8868, -0.1065, -0.3374, -0.5472],
        [ 0.0138, -0.1285, -0.7495, -0.2676,  0.1638,  0.1955, -0.2730,  0.0480,
         -0.0216,  0.1833, -0.1482,  0.2012, -0.1058, -0.2655, -0.2031, -0.4648,
         -0.5155, -0.2494, -0.2936, -0.1638]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1192, 0.0855, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.2470, 0.0089, 0.0384, 0.0000, 0.0000, 0.2871, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.1636, 0.0000, 0.0000, 0.1213, 0.0000, 0.0000, 0.0000, 0.0130,
         0.1749, 0.0000, 0.2043, 0.0000, 0.0000, 0.53

#### nn.Sequential

- nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a networks like `seq_modules`.

In [23]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10),
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)
print(logits.shape)

tensor([[ 0.0814,  0.0685,  0.0060, -0.0571, -0.0558, -0.1367, -0.0659, -0.0836,
         -0.1954,  0.1114],
        [ 0.0119,  0.0637,  0.1038, -0.0762, -0.0494, -0.1387, -0.0118, -0.0727,
         -0.1550,  0.0983],
        [-0.0545, -0.0026,  0.1999, -0.1464, -0.0529, -0.1576, -0.1144, -0.1902,
         -0.1344,  0.0993]], grad_fn=<AddmmBackward0>)
torch.Size([3, 10])


#### nn.Softmax

- last linear layer of the neural network return logits- raw values in (-infty, +infty) which are passes to the `nn.Softmax` module.
- values are normalized between [0,1] representing the model's predicted probabilities for each class. 
- dim parameter indicates the dimension along which the values must sum to 1.

In [24]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.1116, 0.1102, 0.1035, 0.0972, 0.0973, 0.0897, 0.0963, 0.0946, 0.0846,
         0.1150],
        [0.1031, 0.1086, 0.1130, 0.0944, 0.0970, 0.0887, 0.1007, 0.0948, 0.0873,
         0.1124],
        [0.0994, 0.1047, 0.1282, 0.0906, 0.0995, 0.0896, 0.0936, 0.0868, 0.0917,
         0.1159]], grad_fn=<SoftmaxBackward0>)


#### Model Parameters

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods.

In [25]:
print(f"Module Structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n ")

Module Structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0186,  0.0005, -0.0284,  ..., -0.0063, -0.0351,  0.0296],
        [ 0.0214,  0.0002,  0.0188,  ...,  0.0092,  0.0205,  0.0267]],
       device='mps:0', grad_fn=<SliceBackward0>)
 
Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0259,  0.0276], device='mps:0', grad_fn=<SliceBackward0>)
 
Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0223,  0.0098, -0.0209,  ..., -0.0248, -0.0036, -0.0035],
        [ 0.0225,  0.0022,  0.0355,  ...,  0.0186,  0.0116,  0.0373]],
       device='mps:0', grad_fn=<SliceBa